In [ ]:
## This is a short demonstration of some of the things
## python can do for you, and a preview to some of the
## things we'll learn in the course

print('I\'m ready for my life to be improved')

In [ ]:
## Lets import some modules we'll need later!
## This might take a minute. You can tell the cell is still
## working because of the little [*] next to the 'ln' symbol 
## to the left.

import os
import subprocess
import math
import pandas
import nipype
from glob import glob
from scipy import stats,linalg
from scipy.io import savemat
import numpy as np
import nibabel as ni
import networkx as nx
from matplotlib import pyplot,patches


In [ ]:
################ PART 1: COMMAND LINE ###################
## First, I'll show you a bit about how python can help you automate
## command line functions

# let's create a few empty files for demonstration purposes
print('here is our directory before creating the files. . . \n')
print(os.listdir())
for i in range(10):
    os.system('touch empty_fl_%s'%i)
print('\n . . . and after')
os.listdir()

In [ ]:
# let's say we want to rename these files given a list of subject IDs

# generate a list of random subject IDs
ids = np.random.randint(1000,5000,10)
print('here is a list of the IDs we\'ve generated')
print(sorted(ids))
print('\n')

# collect files that we created
fls = sorted(glob(os.path.join(os.getcwd(),'empty_fl*')))
print('here are the files')
print(fls)
print ('\n')

# rename files using subject IDs
for i,fl in enumerate(fls):
    pth,flnm = os.path.split(fl)
    fl_parts = flnm.split('_')
    os.system('mv %s %s'%(fl,os.path.join(pth,'%s_%s_s%s'%(fl_parts[0],fl_parts[1],ids[i]))))

# now we'll list contents again to see if the files were renamed
print('here are the directory contents')
os.listdir()

In [ ]:
# now lets get rid of those files.. we don't need them
os.system('rm empty*')

print('directory contents')
os.listdir()

In [ ]:
## Python can be used to automate any terminal commands
## This includes neuroimaging tools like fsl. 
## Here, we will extract average effect (t) values for three 
## different subjects within a set of 7 masks taken from an 
## atlas of functional brain networks

# collect subjects and atlas
stuff_dir = os.path.join(pth,'stuff')
subs = sorted(glob(os.path.join(stuff_dir,'rw*')))
msk = os.path.join(stuff_dir,'networks_scale7.nii.gz')
print('here is a list of our subjects')
print(subs)
print('\n and our atlas')
print(msk)



In [ ]:
# lets also make a dataframe to collect the values
sub_ids = []
for sub in subs:
    sid = os.path.split(sub)[1].split('.')[0]
    sub_ids.append(sid)

cols = []
for i in range(7):
    cols.append('network_%s'%i)
 
df = pandas.DataFrame(index=sub_ids,columns=cols)
print(df)

In [ ]:
# now lets actually perform the command

# go to the directory
os.chdir(stuff_dir)

# for each network
for i in range(7):
    # make mask of network using fslmaths
    os.system('fslmaths %s -thr %s -uthr %s msk%s'%(msk,(i+1),(i+1),i)) 
    # for each subject
    for x,sub in enumerate(subs):
        # mask subject with mask using fslmaths
        os.system('fslmaths %s -mas msk%s.nii.gz mskd_img'%(sub,i))
        # get mean within mask by reading output from fslstats command
        val = float(subprocess.check_output('fslstats mskd_img.nii.gz -M',shell=True))
        # print value to keep track of progress
        print('the mean value for subject %s in network %s is %s'%(sub_ids[x],i,val))
        # write to DataFrame
        df.ix[sub_ids[x],'network_%s'%(i)] = val
# get rid of scrap
os.system('rm msk*.ni*')

print('\n and here is our dataframe again with the values filled in \n')
print(df)
    

In [ ]:
######### PART 2: SPREADSHEETS ############
## Using pandas lets us open spreadsheets with ease

# let's turn that dataframe into an actual file that can be read
# what are our options?
#df.to_

df.to_excel('networks.xls')
os.listdir()

# we can also read in files of almost any text type using pandas
ndf = pandas.ExcelFile('networks.xls').parse('Sheet1')

print(ndf)



In [ ]:
# there are many nice features of pandas
print('slicing is easy: \n')

print('all networks for subject %s'%(sub_ids[0]))
print(ndf.ix[sub_ids[0]],'\n')
print('all subject values for network 0')
print(ndf.ix[:,cols[0]],'\n')

print('so is indexing: \n')
print('value for subject %s, network 4'%(sub_ids[1]))
print(ndf.ix[sub_ids[1],cols[4]],'\n')

print('there also many things you can do to summarize or manipulate individual slices \n')
slc = ndf.ix[:,cols[0]]
print('here is the mean value across all networks for subject 0 \n')
print(slc.mean())

In [ ]:
########## PART 3: MATH AND STATS ############
## Python has an array of different libraries available for performing 
## mathematical operations. This ranges from simple mathematical commmands
## to basic stats like regression/t-tests, to complex deep learning
## algorithms such as clustering and support vector machines.

# lets make z-scores for each network from the data we just produced

# first we'll make a quick z-score function
def z_score(val,mn,sd): 
    scr = (val - mn) / sd
    return scr

# next, we'll get the means and SDs of each network and store them
# in a dictionary
net_stz = {}
for i in range(7):
    slc = df[:]['network_%s'%(i)]
    net_stz.update({'network_%s'%(i): (slc.mean(),slc.std())})
print('here are the means and sds for each network \n')
print(net_stz,'\n')
    
# now we'll apply it to each subject's data
for sub in df.index.tolist():
    for col in cols:
        val = df.ix[sub,col]
        df.ix[sub,'z_%s'%(col)] = z_score(val,net_stz[col][0],net_stz[col][1])

print('here are the new z-scored values')
print(df[:][df.columns[7:]])
        

In [ ]:
## we can also calculate something slightly more complicated,
## we'll make a fake correlation matrix from scratch, look for
## networks using clustering analysis, and then calculate the 
## some graph theory metrics

# generate random connectivity matrix
gen_mtx = np.random.random((49,49))

# now make its symettrical like a real connectivity matrix
diag = gen_mtx.diagonal()
lo_tri = np.tril(gen_mtx)
up_tri = lo_tri.transpose()
mtx = lo_tri + up_tri
for i in range(49):
    mtx[i][i] = diag[i]
    
print('here is our matrix \n',mtx)
print('\n and here is what it looks like visualized')
pyplot.imshow(mtx,
              cmap="Greys",
              interpolation="none")
pyplot.show()
    




In [ ]:
# and lets make use partial correlations to make it slightly more sparse
def partial_corr(C):
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr
        
    return P_corr

pc_mtx = partial_corr(mtx)
for i in range(49):
    pc_mtx[i][i] = diag[0]

print('now here\'s the new matrix!')
pyplot.imshow(pc_mtx,
              cmap="Greys",
              interpolation="none")
pyplot.show()
        


In [ ]:
# Let's make some sense out of this nonsense. We'll run clustering
# analysis on our random connectivity matrix

from sklearn.cluster import KMeans
# run clustering analysis and get labels
kmeans = KMeans(n_clusters=7, random_state=0).fit(pc_mtx)
labz = kmeans.labels_.tolist()
# reorder connectivity matrix by cluster labels
naxis = []
for x in range(7):
    for i in range(49):
        if labz[i] == x:
            naxis.append(i)
npc_mtx = pc_mtx[:, naxis][naxis]

print('and here is our matrix reordered by cluster membership')
pyplot.imshow(npc_mtx,
              cmap="Greys",
              interpolation="none")
pyplot.show()
print('wow, doesn\'t it look like there is something real happening here?')

In [ ]:
## We could do the graph theory metrics by hand, but we don't
## have to. There are specialized python functions that do the 
## dirty work for you for almost anything you could think of,
## and graph theory is no exception

## first lets convert this to a binarized matrix where we save
## only the top 10% of connections

flat = np.tril(pc_mtx).flatten()
flat = [abs(x) for x in flat]
top_no = int(len(flat) * 0.90)
top_10 = sorted(flat)[top_no]
bin_mtx = np.full((49,49),np.nan)
for i in range(49):
    for j in range(49):
        if pc_mtx[i][j] > top_10:
            bin_mtx[i][j] = 1
        else:
            bin_mtx[i][j] = 0
nbin_mtx = bin_mtx[:, naxis][naxis]
pyplot.imshow(nbin_mtx,
              cmap="Greys",
              interpolation="none")
pyplot.show()
print('as this is random data, the binarized matrix is not very pretty')

# but assuming we had a real matrix it would look better
# Lets run some graph metrix. Choose your poison

G = nx.from_numpy_matrix(bin_mtx)
nx.degree(G)


In [ ]:
## we can also run more traditional statistics like correlations
## lets generate some fake data and run a correlation

# here's some fake data
x = np.linspace(0, 1, 100)
y = (np.cos(x) + 0.5*np.random.rand(100)) + np.random.ranf()
#y = (x + np.random.rand(100)) - np.random.ranf()
p = np.poly1d(np.polyfit(x, y, 1))
t = np.linspace(0, 1, 100)
pyplot.plot(x, y, 'o', t, p(t), '-')
pyplot.show()

# lets test it!
r,pval = stats.pearsonr(x,y)
print('r = %s, p = %s'%(r,pval))

In [ ]:
## That's all well and good, but with Python, we can add another level
## of sophistication through automation: random sampling methods

## Let's generate some fake data again.
## But this time, lets make the correlation less tight
## And to show off the versatility of python, let's use completely different
## commands to generate the data and to plot

# generate fake data
xx = np.array([-1, 1])
yy = np.array([-1.5, 1.5])
means = [xx.mean(), yy.mean()]  
stds = [xx.std() / 3, yy.std() / 3]
corr = 0.3         # correlation
covs = [[stds[0]**2          , stds[0]*stds[1]*corr], 
        [stds[0]*stds[1]*corr,           stds[1]**2]] 

# get plotting paramaters
ms = np.random.multivariate_normal(means, covs, 50).T
slp,inter = np.polyfit(ms[0],ms[1],1)

# plot
pyplot.plot(ms[0],ms[1], 'o')
pyplot.plot(ms[0],slp*ms[0]+inter,'-')

pyplot.show()

# run stats
r,pval = stats.pearsonr(ms[0],ms[1])
print('r = %s, p = %s'%(r,pval))

In [ ]:
## let's say we wanted to resample the data to establish exact p-values
## we can shuffle x (with replacement) over 10,000 permutations 

## let's also time it..
import timeit

distr = []
start_time = timeit.default_timer()
for i in range(10000):
    new_x = np.random.permutation(ms[0].tolist())
    new_r,new_p = stats.pearsonr(new_x,ms[1])
    distr.append(new_r)
elapsed = timeit.default_timer() - start_time

print('permuted %s samples in %s seconds \n'%(len(distr),elapsed))# 

pyplot.hist(distr)
pyplot.title("Null distribution of r-values")
pyplot.xlabel("r-value")
pyplot.ylabel("Frequency")


print('here is our distribution of r-values')
pyplot.show()



In [ ]:
# now lets find our p-value and test the hypothesis that the
# correlation did not occur from chance

# assess null distribution to establish p-value
slicer = []
for v in sorted(distr):
    if v > r:
        slicer.append(v)
exact_p = len(slicer)/float((len(distr)+1))


# what r-value is p<0.05?
sig_r = sorted(distr)[int((1-0.05) * len(distr))]
# what r-value is p<0.025 (two tailed)
sig_r2 = sorted(distr)[int((1-0.025) * len(distr))]

# let's have a look
print('the exact p-value is %s \n'%(exact_p))
print('the black line is the observed r-value')
print('the dotted red line represents p<0.05 (one tailed)')
print('the solid red line represents p<0.025 (two tailed)')

pyplot.hist(distr,color='blue')
pyplot.title("Null distribution of r-values")
pyplot.xlabel("r-value")
pyplot.ylabel("Frequency")
pyplot.axvline(x=sig_r,color='red',linestyle='dashed')
pyplot.axvline(x=r,color='black')
pyplot.axvline(x=sig_r2,color='red')
pyplot.show()

print(exact_p)
print(sig_r)
print(sig_r2)

In [ ]:
######## PART 4: Loading, saving and manipulating images ###########
## None of this would be much good if you couldn't use it to deal with
## actual imaging data. Luckily, python can handle that too.

# let's take the first subject scan from earlier
print('woring on subject %s'%(subs[0]))
img = ni.load(subs[0])

print('now we have access to the affine... \n')
print(img.affine)
print('\n and the header...\n')
print(img.header)
print('\n and the actual image data, which is a matrix of shape %s,%s,%s \n'%(img.shape[0],
                                                                                img.shape[1],
                                                                                img.shape[2]))
print(img.get_data())

#print('\n',img.get_data()[:][40][40])

In [ ]:
## You can perform analyses on these matrices just like you would any other
## and can easily save the data back into a new images.
## Just as a simple example, we'll threshold each image so that only top 5%
## of non-zero values of each image shown, and then we'll binarize it to make 
## a mask.

#and lets time it!
start_time = timeit.default_timer()

for sub in subs:
    # get data and affine (for later)
    s_img = ni.load(sub)
    s_data = s_img.get_data()
    s_aff = s_img.affine
    sx,sy,sz = s_img.shape
    # figure out 5% threshold for non-zero values
    s_distr = []
    for i in range(sx):
        for j in range(sy):
            for k in range(sz):
                if s_data[i][j][k] > 1:
                    s_distr.append(s_data[i][j][k])
    thr = sorted(s_distr)[int(len(s_distr) * 0.95)]
    print('value threshold for subject %s is %s'%(sub,thr))
    # binarize image
    s_bin = np.where(s_data>thr,1,0)
    # save new binarized, thresholded image
    pth,flnm = os.path.split(sub)
    nflnm = os.path.join(pth,'%s_p05mask.nii.gz'%(flnm.split('.')[0]))
    n_img = ni.Nifti1Image(s_bin, s_aff)
    n_img.to_filename(nflnm)
    print('image written to %s \n'%(nflnm))

elapsed = timeit.default_timer() - start_time
print('\n manipulation performed on 3 images in %s seconds'%(elapsed))
print('assuming linear scaling, that would mean 300 subjects would take %s minutes'%((100*elapsed)/60))


In [ ]:
## Some poor souls hang nostalgically onto MatLab, believing
## everything on earth is best represented my matrices, and that
## all things should be followed by a ;
## Luckily for those poor souls, we can easily export anything to a
## .mat file that is compatible with matlab.

# We still have our matrix from before. Let's make it into a matfile

# First convert to a dictionary
mat_dict = {'fake_matrix': pc_mtx}

# Save the file
savemat('new_matfile',mat_dict)

os.listdir()

In [ ]:
import nipype.interfaces.fsl as fsl

In [ ]:
fsl.preprocess.FLIRT?

In [ ]:
subs = glob(os.path.join(pth,'w*'))
ref = '/Users/jakevogel/bellec_lab/aaic/rMNI152_T1_2mm_brain.nii'
flt = fsl.preprocess.FLIRT()
for sub in subs:
    flt.inputs.in_file = sub
    flt.inputs.reference = ref
    flt.inputs.output_type = 'NIFTI_GZ'
    flt.inputs.interp = 'nearestneighbour'
    flt.cmdline
    flt.run()
#flt = fsl.FLIRT()
#flt.inp


In [ ]:
os.listdir()

In [ ]:
# clean up
os.system('rm *p05*')
os.system('rm *flirt*')
os.system('rm *.mat')
os.system('rm *xls')